In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import Select   # seleccion de un dropdown


import time

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')


import asyncio

from tqdm.notebook import tqdm
from joblib import Parallel, delayed

In [2]:
from unidecode import unidecode 

In [3]:
url = 'https://www.mundodeportivo.com/resultados/futbol/laliga'

In [4]:
driver = webdriver.Chrome()

driver.get('https://www.mundodeportivo.com/resultados/futbol/laliga')

In [5]:
aceptar = driver.find_element(By.XPATH, '//*[@id="didomi-notice-agree-button"]')

aceptar.click()

In [6]:
aceptar_1 = driver.find_element(By.XPATH, '//*[@id="gg-alert-buttons"]/a[2]')

aceptar_1.click()

In [7]:
clasificación =driver.find_element(By.CSS_SELECTOR, 'body > header > div.header-menu-container > div.container.swiper-header-menu-container > div > div > ul > li:nth-child(3) > a')
clasificación.click()


In [8]:
dropdown = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[1]/div/div[2]')
dropdown.click()



In [9]:
fecha = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[1]/div/div[2]/div/a[2]')
fecha.click()

In [27]:
#encuentra la tabla en la web con el path
tabla_elemento = driver.find_element(By.XPATH, '//*[@id="main-container"]/div[3]/div/div[1]/div[2]/div/table')

# encuentra cada fila de la tabla
filas = tabla_elemento.find_elements(By.TAG_NAME, "tr")
data = []

# iterable por todos los datos de la tabla
for fila in filas:
    columnas = fila.find_elements(By.TAG_NAME, "td")
    fila_data = [columna.text for columna in columnas]
    data.append(fila_data)

In [28]:
df = pd.DataFrame(data, columns=None)

In [29]:
df.head()

,0,1,2,3,4,5,6,7
0,EQUIPOS,PJ,PG,PE,PP,GF,GC,PTS
1,1\nBARCELONA,38,28,4,6,70,20,88
2,2\nREAL MADRID,38,24,6,8,75,36,78
3,3\nATLÉTICO DE MADRID,38,23,8,7,70,33,77
4,4\nREAL SOCIEDAD,38,21,8,9,51,35,71


In [30]:
# convierte la primera fila en las columnas 
nombres_columnas = df.iloc[0]
# reseteo el index
nuevo_df = df[1:].reset_index(drop=True)
nuevo_df.columns = nombres_columnas

In [33]:
nuevo_df.head()

,EQUIPOS,PJ,PG,PE,PP,GF,GC,PTS
0,1\nBARCELONA,38,28,4,6,70,20,88
1,2\nREAL MADRID,38,24,6,8,75,36,78
2,3\nATLÉTICO DE MADRID,38,23,8,7,70,33,77
3,4\nREAL SOCIEDAD,38,21,8,9,51,35,71
4,5\nVILLARREAL,38,19,7,12,59,40,64


In [36]:
nuevo_df.columns

Index(['EQUIPOS', 'PJ', 'PG', 'PE', 'PP', 'GF', 'GC', 'PTS'], dtype='object', name=0)

In [37]:
# lambda para splitear por cada uno de los elementos de la ccolumna equipos
nuevo_df['EQUIPOS'] = nuevo_df['EQUIPOS'].apply(lambda x: x.split('\n', 1)[-1])

In [38]:
nuevo_df.head()

,EQUIPOS,PJ,PG,PE,PP,GF,GC,PTS
0,BARCELONA,38,28,4,6,70,20,88
1,REAL MADRID,38,24,6,8,75,36,78
2,ATLÉTICO DE MADRID,38,23,8,7,70,33,77
3,REAL SOCIEDAD,38,21,8,9,51,35,71
4,VILLARREAL,38,19,7,12,59,40,64


In [40]:
nuevo_df['EQUIPOS'] = nuevo_df['EQUIPOS'].apply(unidecode)

In [41]:
nuevo_df.head()

,EQUIPOS,PJ,PG,PE,PP,GF,GC,PTS
0,BARCELONA,38,28,4,6,70,20,88
1,REAL MADRID,38,24,6,8,75,36,78
2,ATLETICO DE MADRID,38,23,8,7,70,33,77
3,REAL SOCIEDAD,38,21,8,9,51,35,71
4,VILLARREAL,38,19,7,12,59,40,64


In [42]:
# para poner la primera letra mayuscula
nuevo_df['EQUIPOS'] = nuevo_df['EQUIPOS'].apply(lambda x: x.title())

In [43]:
nuevo_df.head()

,EQUIPOS,PJ,PG,PE,PP,GF,GC,PTS
0,Barcelona,38,28,4,6,70,20,88
1,Real Madrid,38,24,6,8,75,36,78
2,Atletico De Madrid,38,23,8,7,70,33,77
3,Real Sociedad,38,21,8,9,51,35,71
4,Villarreal,38,19,7,12,59,40,64


In [44]:
nuevo_df.columns = ['equipos', 'partidos jugados', 'partidos ganados','partidos empatados','partidos perdidos','goles a favor','goles en contra','total puntos']

In [46]:
# nueva columna con un numero que empieza en 1 y se le va sumando 1 hasta la longitud del dataframe
nuevo_df['ID_equipo'] = range(1, len(nuevo_df) + 1)

In [47]:
nuevo_df.head()

,equipos,partidos jugados,partidos ganados,partidos empatados,partidos perdidos,goles a favor,goles en contra,total puntos,ID_equipo
0,Barcelona,38,28,4,6,70,20,88,1
1,Real Madrid,38,24,6,8,75,36,78,2
2,Atletico De Madrid,38,23,8,7,70,33,77,3
3,Real Sociedad,38,21,8,9,51,35,71,4
4,Villarreal,38,19,7,12,59,40,64,5


In [48]:
columnas_ordenadas = ['ID_equipo', 'equipos', 'partidos jugados', 'partidos ganados',
                       'partidos empatados', 'partidos perdidos', 'goles a favor',
                       'goles en contra', 'total puntos']

nuevo_df = nuevo_df[columnas_ordenadas]

In [1]:
nuevo_df.head()

NameError: name 'nuevo_df' is not defined